In [2]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from utils.dataset import CandyDataset

2024-09-08 20:55:59.640437: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-08 20:55:59.695245: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-08 20:55:59.696204: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-08 20:56:00.615094: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
base_dir = os.getcwd()
dataset_dir = os.path.join(base_dir, 'dataset', '3 - normalized')

dataset = CandyDataset(dataset_dir)
dataset.split_dataset()

In [4]:
train_set_data = dataset.get_images_data(dataset.data_split['train'])
test_set_data  = dataset.get_images_data(dataset.data_split['test'])
validation_set_data  = dataset.get_images_data(dataset.data_split['val'])

train_images = np.array([d['image'] for d in train_set_data])  / 255.0
train_labels = np.array([d['category_id'] - 1 for d in train_set_data])

test_images = np.array([d['image'] for d in test_set_data])  / 255.0
test_labels = np.array([d['category_id'] - 1 for d in test_set_data])

validation_images = np.array([d['image'] for d in validation_set_data])  / 255.0
validation_labels = np.array([d['category_id'] - 1 for d in validation_set_data]) 


# Convertendo os rótulos para one-hot encoding
num_classes = len(np.unique(train_labels))  # Assumindo que o número de classes seja igual para todos os conjuntos

train_labels = to_categorical(train_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)
validation_labels = to_categorical(validation_labels, num_classes)


In [5]:
kernel = (5, 5)
pool = (2, 2)
num_classes = 10

model = Sequential([
    Conv2D(16, kernel, padding='same', activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D(pool_size=pool),
    Conv2D(32, kernel, padding='same', activation='relu'),
    MaxPooling2D(pool_size=pool),
    Conv2D(64, kernel, padding='same', activation='relu'),
    MaxPooling2D(pool_size=pool),
    Conv2D(128, kernel, padding='same', activation='relu'),
    MaxPooling2D(pool_size=pool),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.summary()

2024-09-08 21:04:37.467997: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-08 21:04:37.469185: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 16)      1216      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      12832     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        51264     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 32, 32, 64)       0

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_images, train_labels, validation_data=(validation_images, validation_labels), epochs=10)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc}")